# AD9833 ToolBox test with FTDI
https://www.analog.com/media/en/technical-documentation/data-sheets/ad9833.pdf

In [1]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [2]:
# paths = ['']
# paths = [gen_relative_path(p) for p in paths]
# print(paths)

In [3]:
paths = [['..', '..', '..', '..', 'ORM', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', '..', 'USB', 'Universal Serial Bus', 'bitbucket', 'github', 'codes'], 
         ['..', '..', '..', '..', '..', '已完成', 'Bridges', 'bitbucket', 'github', 'codes'], 
         ['..', 'codes']]

for path in paths:
    append_source_relative_path(path)

In [4]:
%pylab inline 

from signal_generators import adapters, tools
from signal_generators.ad98xx.ad9833 import AD9833
from signal_generators.shift_register import ShiftRegister

Populating the interactive namespace from numpy and matplotlib


## Debug mode?

In [5]:
adapters.SPI.DEBUG_MODE = False  # whether to show SPI written data.
AD9833.DEBUG_MODE = False        # whether to dump registers. 

## Generators

In [6]:
with_hardware_device = True

if with_hardware_device:
    _clk = adapters.Pin.get_Ftdi_pin(pin_id = 4)
    _data = adapters.Pin.get_Ftdi_pin(pin_id = 1)

    _ss1 = adapters.Pin.get_Ftdi_pin(pin_id = 3)
    _ss2 = adapters.Pin.get_Ftdi_pin(pin_id = 0)

    _spi_1 = ShiftRegister(stb_pin = _ss1, clk_pin = _clk, data_pin = _data, polarity = 1)
    _spi_2 = ShiftRegister(stb_pin = _ss2, clk_pin = _clk, data_pin = _data, polarity = 1)
else:
    _spi_1 = _spi_2 = _ss1 = _ss2 = None  # using None for testing without actual hardware device.

ad1 = AD9833(_spi_1, _ss1)
ad2 = AD9833(_spi_2, _ss2)

In [7]:
ad1.reset()
ad2.reset()

In [8]:
ad1.enable_output(False)
ad2.enable_output(False)

In [9]:
tb = tools.ToolBox()

## Frequency Sweeping

In [10]:
tb.sweep(ad1, freq_start = 10, freq_end = int(1e6), n_freqs = 100,
         sweep_type = 'logarithm', direction = 'round_trip', n_cycles = 2,
         slot_duration = 0.01, between_cycle_seconds = 0)

Frequency:      10.00
Frequency:      11.22
Frequency:      12.59
Frequency:      14.13
Frequency:      15.85
Frequency:      17.78
Frequency:      19.95
Frequency:      22.39
Frequency:      25.12
Frequency:      28.18
Frequency:      31.62
Frequency:      35.48
Frequency:      39.81
Frequency:      44.67
Frequency:      50.12
Frequency:      56.23
Frequency:      63.10
Frequency:      70.79
Frequency:      79.43
Frequency:      89.13
Frequency:     100.00
Frequency:     112.20
Frequency:     125.89
Frequency:     141.25
Frequency:     158.49
Frequency:     177.83
Frequency:     199.53
Frequency:     223.87
Frequency:     251.19
Frequency:     281.84
Frequency:     316.23
Frequency:     354.81
Frequency:     398.11
Frequency:     446.68
Frequency:     501.19
Frequency:     562.34
Frequency:     630.96
Frequency:     707.95
Frequency:     794.33
Frequency:     891.25
Frequency:    1000.00
Frequency:    1122.02
Frequency:    1258.93
Frequency:    1412.54
Frequency:    1584.89
Frequency:

Frequency:     141.25
Frequency:     125.89
Frequency:     112.20
Frequency:     100.00
Frequency:      89.13
Frequency:      79.43
Frequency:      70.79
Frequency:      63.10
Frequency:      56.23
Frequency:      50.12
Frequency:      44.67
Frequency:      39.81
Frequency:      35.48
Frequency:      31.62
Frequency:      28.18
Frequency:      25.12
Frequency:      22.39
Frequency:      19.95
Frequency:      17.78
Frequency:      15.85
Frequency:      14.13
Frequency:      12.59
Frequency:      11.22
Frequency:      10.00


[10.0,
 11.220184543019636,
 12.589254117941675,
 14.12537544622754,
 15.848931924611133,
 17.78279410038923,
 19.952623149688797,
 22.3872113856834,
 25.118864315095795,
 28.183829312644534,
 31.622776601683793,
 35.48133892335755,
 39.810717055349734,
 44.6683592150963,
 50.11872336272725,
 56.23413251903491,
 63.09573444801933,
 70.79457843841381,
 79.43282347242814,
 89.12509381337459,
 100.0,
 112.2018454301963,
 125.89254117941675,
 141.25375446227554,
 158.48931924611142,
 177.82794100389228,
 199.52623149688787,
 223.872113856834,
 251.18864315095823,
 281.8382931264455,
 316.22776601683796,
 354.8133892335753,
 398.1071705534973,
 446.6835921509635,
 501.18723362727246,
 562.341325190349,
 630.957344480193,
 707.945784384138,
 794.3282347242822,
 891.2509381337459,
 1000.0,
 1122.018454301964,
 1258.9254117941675,
 1412.537544622754,
 1584.893192461114,
 1778.2794100389228,
 1995.262314968881,
 2238.72113856834,
 2511.886431509582,
 2818.382931264455,
 3162.2776601683795,
 354

## Function Toggling

In [11]:
duration_seconds, count, cycle_time = tb.toggle(ad1,
                                                fun = 'enable_output', params = ({'value': True}, {'value': False}),
                                                n_cycles = 10, slot_duration = 0.2, between_cycle_seconds = 0.2)
print('duration_seconds, count, cycle_time', duration_seconds, count, cycle_time)

duration_seconds, count, cycle_time 6.262455463409424 10 0.6262455463409424


## Waveform Juggling

In [12]:
tb.juggle((ad1, ad2),
          freq_start = int(1e2), freq_end = int(2e4), n_freqs = 100, freqs_type = 'logarithm',
          slot_duration = 0.2, between_cycle_seconds = 0.2,
          n_juggles = 10)

freq:     105.44	phase:  42.00	shape: triangle
freq:     320.79	phase: 292.00	shape: sine
freq:    1657.85	phase:  20.00	shape: triangle
freq:    1748.06	phase: 201.00	shape: triangle
freq:     638.79	phase: 220.00	shape: sine
freq:     111.18	phase: 186.00	shape: sine
freq:     832.55	phase: 303.00	shape: square
freq:    3130.90	phase: 330.00	shape: sine
freq:     574.57	phase:  58.00	shape: triangle
freq:     259.53	phase: 319.00	shape: half_square
freq:     376.06	phase:  63.00	shape: triangle
freq:   18967.92	phase:  78.00	shape: triangle
freq:    3870.00	phase: 178.00	shape: sine
freq:     748.84	phase:  35.00	shape: triangle
freq:   17989.10	phase: 345.00	shape: half_square
freq:    1491.16	phase:  88.00	shape: sine
freq:     320.79	phase: 233.00	shape: square
freq:   12414.73	phase: 196.00	shape: sine
freq:     418.10	phase: 158.00	shape: sine
freq:    1657.85	phase:  92.00	shape: triangle
